Assignment 1: Crypto
=============

Introduction
------------

In this lab, you will implement and break several cyptographic algorithms or insecure random number generators. 

Please follow this to complete the assignment. You can fill out the blanks as it appears as to be instructed.  
All packages that you may need is already imported. If you are pretty sure that you must use more, please contact the instructor. 


Configuration
-------------

This notebook is written on a Mac, using vscode, and tested on [Google colab](https://colab.research.google.com/).  
Among the modules required, the notebook installs `pycrypto` automatically as shown below,  
and you need to user python version 3.6 or later due to `secret` module.  

We recommend you to either work on somewhere with vscode as most of systems these days have  
python >= 3.6. If this option doesn't work, Google Colab should be a working option.  


Modules that you are allowed to use.
------------------------------------

The building blocks that you have from the packages are:
- base64 encoder/decoder
- AES block cipher with mode of operation implement.
- Secure random token generation.

Please contact the instructor if you are really sure that you must use the modules which are not imported by default.

** Please note that changes to the code blocks other than the ones for you could result in 0 grade for the challenge.**


References
----------

These challenges are originally grepped from [https://cryptopals.com/].  
**YOU MUST NOT JUST BRING THE SOLUTIONS AVAILABLE ONLINE.**  
It will result in cheating.




In [1]:
! pip3 install pycrypto
from secrets import token_bytes, randbelow
from base64 import b64decode, b64encode
from Crypto.Cipher import AES
from time import sleep, time
from Crypto.PublicKey import RSA
from Crypto import Random
from math import gcd
from requests import get
import operator, string

# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

test_input_urls = [
    'https://pastebin.com/raw/wF5qra2w',
    'https://pastebin.com/raw/YLeTER7C'
]
test_inputs = []
if __name__ == '__main__':
    for u in test_input_urls:
        resp = get(u)
        if (resp.status_code != 200):
            print ('failed to fetch from {}. Please report to the instructor.'.format(u))
        test_inputs.append(resp.text)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


(*0 points*) Challenge 0: Data types
-----------------------

In this lab, you will be dealing with three types of data: `int`, `string`, and `bytes`.  
To print out and dealwith the bytes, you may find it useful to encode is with base64.  
For more details, please refer to [a wiki article](https://en.wikipedia.org/wiki/Base64).

The following is some examples of playing with the types.


In [2]:
string_t = 'I hate security.'
print(string_t)
byte_t = string_t.encode('utf-8')
print(byte_t)
b64_t = b64encode(byte_t)
print(b64_t)
int_t = int.from_bytes(byte_t,'big')
print(int_t)
print(format(int_t,'x'))
int_t = int.from_bytes(b64_t,'big')
print(int_t)
print(format(int_t,'x'))


I hate security.
b'I hate security.'
b'SSBoYXRlIHNlY3VyaXR5Lg=='
97201914283525032754921666349643364654
4920686174652073656375726974792e
2043128755156558113254692191299997088232814889420729040189
5353426f5958526c49484e6c59335679615852354c673d3d


(*30 points*) Challenge 1: Breaking XOR encryptions.
---------------------------------------------------

### Description

In this challenge, you will be given a cipher text that is encrypted with a fixed (unknown)-length XOR enrtypion.
`get_xor_encryption oracle` gives you a pair or function that encrypts and decrypts the given input bytes.
`test_xor` shows an example of using it.


In [3]:
def get_xor_encryption_oracle_1(key_size = 16, key = None):
    if (key == None):
        key = token_bytes(key_size)
    def xor_bytes(key_bytes: bytes,val_bytes: bytes):
        cipher = b''
        key_len = len(key_bytes)
        for i in range(len(val_bytes)):
            k = key_bytes[i%key_len]
            v = val_bytes[i]
            cipher += (key_bytes[i%key_len] ^ val_bytes[i]).to_bytes(1,'big')
        return cipher
    def encrypt(plain: bytes):
        return xor_bytes(key,plain)
        pass
    def decrypt(cipher: bytes):
        return xor_bytes(key,cipher)
        pass
    
    return encrypt, decrypt

def test_xor():
    input_str = 'I hate cryptography.'
    input_bytes = input_str.encode('utf-8')
    enc,dec = get_xor_encryption_oracle_1()
    cipher = enc(input_bytes)
    plain = dec(cipher)
    plain_str = plain.decode('utf-8')
    print('input_str: ' + input_str)
    print('cipher:    ' + b64encode(cipher).decode('utf-8'))
    print('plain_str: ' + plain_str)
    print(plain_str == input_str)



if __name__ == '__main__':
    test_xor()




input_str: I hate cryptography.
cipher:    jwwq4zKOk0i+oDw9hucYe7ZEO6w=
plain_str: I hate cryptography.
True


### Task

Your task is to fill out `crack_xor` that returns plain text as a string from an xor-encrypted ciphertext.  
Grader will use a function like `test_crack_xor` to test your submission.  
Please note the following.
1. Assume that the key size is 16.
2. `crack_xor` cannot use the oracle (any function that the oracle gives you). The oracle implementation is just for you to test your crack.
3. Assume that the plaintext is a long text in English (the statistics-based approach in Lec 3 works!).
4. Please make sure that your cracker successfully decrypts the `test_input` given above.


#### Strategy

1. Note that `get_xor_encryption_oracle_1` is configurable.  
2. Please first try to crack when key size is 1. When key size is 1, it's just substitution.  
3. Using that, as you know the key size and the ECB mode is being used, you can collect the bytes encrypted with each byte in the key. Apply the function above.
4. Please remember that the real sentences also have quite a lot of newlines and spaces.


In [4]:
"""
The idea to break by one bytes.
When we key size one byte, we can easily check 256 variants and choose the one with the most engligh letters.
And we apply the same logic for the 16 key size, just by breaking it. So, divide and conquer!!!
- Erkin Matkaziev (20152032)
"""
class Tools_1:
    pass

def xor_bytes(key_bytes: bytes,val_bytes: bytes):
        cipher = b''
        key_len = len(key_bytes)
        for i in range(len(val_bytes)):
            k = key_bytes[i%key_len]
            v = val_bytes[i]
            cipher += (key_bytes[i%key_len] ^ val_bytes[i]).to_bytes(1,'big')
        return cipher

def isOkay(str):
    countMe = 0
    for i in range(len(str)):
        if (str[i] >= 127 or chr(str[i]) == '/' or chr(str[i]) == '*' or chr(str[i]) == '%' or chr(str[i]) == '#' or chr(str[i]) == '<' or chr(str[i]) == '&' or chr(str[i]) == '+' or chr(str[i]) == '-'):
            return False
        if (str[i] < 10 or chr(str[i]) == '&' or (str[i] >= 48 and str[i] <= 62) or str[i] == 33 or chr(str[i]) == '`'):
            return False
        if (str[i] == 63):
            countMe += 1
        if (countMe >= 4):
            return False
    return True

def countLetters(str):
    sum = 0
    for i in range(len(str)):
        if (chr(str[i]).isalpha()):
             sum += 1
    return sum

def countBig(str):
    sum = 0
    for i in range(len(str)):
        if (str[i] >= 65 and str[i] <= 90):
             sum += 1
    return sum

def countSmall(str):
    sum = 0
    for i in range(len(str)):
        if (str[i] >= 97 and str[i] <= 122):
             sum += 1
    return sum

def countSymbol(str):
    sum = 0
    for i in range(len(str)):
        if (chr(str[i]).isalpha()):
             sum += 0
        else:
            sum += 1
    return sum

def crackXorOne(cipher: bytes):
    maxiLetter = 0
    minSymbol = -1
    bestAnswer = 0
    for key in range(0, 256):
        byteKey = key.to_bytes(1, 'big')
        testStr = xor_bytes(byteKey, cipher)
        ###  Use better method
        if (isOkay(testStr) == False):
            continue
        countMe = 0
        for i in range(len(testStr)):
            if ((testStr[i] >= 97 and  testStr[i] <= 127) or (testStr[i] >= 65 and testStr[i] == 90) or testStr[i] == 32):
                countMe += 1
        if ((countMe * 1.0) / len(testStr) > 0.8):
            bestAnswer = testStr
    return bestAnswer

def crack_xor(cipher: bytes):
    key = b''
    batches = [b''] * 16
    for i in range(len(cipher)):
        batches[i % 16] += cipher[i].to_bytes(1, 'big')
    for j in range(len(batches)):
        batches[j] = crackXorOne(batches[j])
    for i in range(100000):
        isBreak = False
        for j in batches:
            if (i < len(j)):
                key += j[i].to_bytes(1, 'big')
                isBreak = True
        if (isBreak == False):
            break
    return key

def test_crack_xor():
    enc, dec = get_xor_encryption_oracle_1(16)
    for i in range(len(test_inputs)):
        t = test_inputs[i]
        cipher = enc(t.encode('utf-8'))
        print(crack_xor(cipher).decode('utf-8'))
        print('test {}: '.format(i) + str(dec(cipher).decode('utf-8') == crack_xor(cipher).decode('utf-8')))
        pass
    pass

test_crack_xor()
    
    
    

Help me, I've fallen on the inside
I tried to change the game
I tried to infiltrate, but now I'm losing
Men in cloaks always seem to run the show
Save me from the
Ghosts and shadows before they eat my soul
Mercy
Mercy
Show me mercy, from the powers that be
Show me mercy, can someone rescue me?
Absent gods and silent tyranny
We're going under, hypnotized by another puppeteer
And tell me why the men in cloaks always have
To bring me down
Running from the
Ghosts and shadows the world just disavows
Mercy
Mercy
Show me mercy, from the powers that be
Show me mercy, can someone rescue me?
Show me mercy
Show me mercy, please
Help me, I've fallen on the inside
And all the men in cloaks trying to devour my soul
Show me mercy from the powers that be
Show me mercy from the gutless and mean
Show me mercy from the killing machines
Show me mercy, can someone rescue me?
test 0: True
The snow glows white on the mountain tonight
Not a footprint to be seen
A kingdom of isolation
And it looks like I'm the

(*30 Points*) Challenge 2: Breaking insecure ECB
----------------------------------

### Descryption

In practice, you might be capable of altering a part of ciphertext that you want to decrypt. i.e.,  


```
CIPHER = AES-128-ECB(your-controlled-bytes || unknown-bytes, unknown-key)
```
(`||` stands for concatenation. (`1 || 2 = 12`))


`get_oracle_2` gives you a pair `encrypt`,`decrypt` doing the above, and `get_unknown_bytes` for you to debug.


In [5]:
def get_oracle_2(unknown_bytes = token_bytes(13), key_size = 16):
    
    
    key = token_bytes(key_size)
    o = AES.new(key,AES.MODE_ECB)
    
    def __pkc7_pad(block,size = 16):
        num_to_pad = (size - (len(block) % size)) % size
        if num_to_pad == 0: num_to_pad = size
        ret = block + bytes([num_to_pad]) * num_to_pad
        return ret

    def __pkc7_unpad(padded: bytes):
        pad = padded[-1]
        suffix = bytes([])
        for i in range(len(padded)):
            if padded[-1-i] == pad:
                suffix = bytes([pad]) + suffix
        if len(suffix) >= pad:
            return padded[:-pad]
        else:
            print('bad padding')
            raise Exception

    def encrypt(plain_prefix: bytes):

        plain = __pkc7_pad(plain_prefix + unknown_bytes)
        return o.encrypt(plain)
        pass
    def decrypt(cipher: bytes):
        plain = __pkc7_unpad(o.decrypt(cipher))
        prefix = plain[:-len(unknown_bytes)]
        return xor_bytes(key,cipher)
        pass
    def get_unknown_bytes():
        return unknown_bytes
    
    return encrypt, decrypt, get_unknown_bytes






### Task

Your task is to fill out `crack_ecb` that reveals the unknown string embedded in an incoming oracle.  
Grader will use a function like `test_crack_ecb` to test your submission.  
Please note the following.

- Assume that the key size is 16.


#### Strategy

1. You can give the oracle an arbitrarily long bytes as an input. What happens if you give 15 bytes? 
2. How long does it take to find out one byte bruteforcely?
3. Can we repeat?



In [6]:
'''
When we create input of size 15 additional byte from random will be added
{input} | random[0]
so to find that random[0]  we create another input with size 16, but last byte will be generated
if we generate right byte our first blocks of cipher will match

so after finding first unknown byte, we then need to inject 2 bytes from unknown
to do we create input of size 14, so last 2 will be from random
{input} | random[0] | randomp[1]
and here for the next input we take create new input with size 16, but with random[0] which we found
and last one will be generated

So we can find each unknown byte one-by-one using bruteforce method, this is using disadavantage of ECB encryption where
when blocks are same they will have same blocks in the cipher

TIME COMPLEXITY: len(unknown) * 256
'''
def crack_ecb(enc_oracle):
    block_size = 16
    # save found bytes here
    found_unknown = b''
    while True:
        prefix_size = 15 - len(found_unknown) # space to insert additional bytes from unknown
        plain = bytes([0]) * prefix_size
        cipher = enc_oracle(plain)
        
        isFound = False
        for j in range(256):
            new_plain = plain + found_unknown + bytes([j]) # this one is we put found bytes and 
                                                           # generate last one which we haven't find yet
                
            new_cipher = enc_oracle(new_plain) # block of cipher match with original cipher block, since they are equal
            
            if (cipher[0:16] == new_cipher[0:16]): # matched
                found_unknown += bytes([j]) # last byte which we found
                isFound = True
                break;
        
        if(isFound == False):
            break
        
    return found_unknown[0:len(found_unknown) - 1]

def test_crack_ecb():
    enc, dec, gus = get_oracle_2()
    res = 'PASS' if crack_ecb(enc) == gus()  else 'FAIL'
    print(res)

if __name__ == '__main__':
    test_crack_ecb()

PASS


(*20 Poiunts*) Challenge 3: Breaking AES CTR under a particular condition
----------------------------------------------------------------------

### Description

Though the counter mode is one of the desirable mode of operation as we discussed in the class,  
we should follow the condition: not resuing (K,IV) pairs. 

As the counter mode enables you to `seek` to a particular byte, you may end of allowing  
the edit of cipher text by bytes.

`oracle_4` gives you and oracle with which you can `encrypt`, `decrypt` and `edit` a ciphertext. 

Note that `encrypt ` and `decrypt` is provided only for debugging purposes.


Though the `edit` does not allow you to decrypt an arbitrary ciphertext, it ends up allow an attacker to decrypt an arbitrary ciphertext.




In [7]:
class oracle_3:

    def __init__(self):
        self.key = token_bytes(16)
        self.iv = token_bytes(16)
        self.i = 0
        self.o = AES.new(self.key,AES.MODE_CTR,counter=self.counter)

    def counter(self):
        self.i += 1
        return (int.from_bytes(self.iv,'big') + self.i).to_bytes(16,'big')
    
    def __pkc7_pad(self,block,size):
        num_to_pad = (size - (len(block) % size)) % size
        if num_to_pad == 0: num_to_pad = size
        ret = block + bytes([num_to_pad]) * num_to_pad
        return ret

    def pkc7_unpad(self, padded: bytes):
        pad = padded[-1]
        suffix = bytes([])
        for i in range(len(padded)):
            if padded[-1-i] == pad:
                suffix = bytes([pad]) + suffix
        if len(suffix) >= pad:
            return padded[:-pad]
        else:
            print('bad padding')
            raise Exception

    def encrypt(self, plain: bytes):
        self.i = 0
        padded = self.__pkc7_pad(plain,16)
        return self.o.encrypt(padded)
        
    def decrypt(self, cipher: bytes):
        self.i = 0
        plain = self.o.decrypt(cipher)
        return self.pkc7_unpad(plain)

    def edit(self, cipher: bytes, offset: int, newtext: bytes):
        self.i = 0
        plain = self.decrypt(cipher)
        len_edit = min(len(newtext), len(plain)-offset)
        prefix = plain[:offset]
        surfix = plain[offset+len_edit:]
        plain = prefix + newtext[:len_edit] + surfix
        return self.encrypt(plain)
    


if __name__ == '__main__':
    o = oracle_3()
    plain = b'I hate cryptography'
    cipher = o.encrypt(plain)
    e = o.edit(cipher,10,b'aaaaa')
    print(o.decrypt(e))
    print(plain)


b'I hate cryaaaaaaphy'
b'I hate cryptography'


### Task

Implement the `crack_by_edit` that takes a ciphertext and an oracle with `edit` (without `encrypt` or `decrypt`) as inputs,  
and returns the plaintext.






In [8]:
'''
In this task we overwrite cipher with zero bytes and encrypt it
Zero bytes encrypted with some key it will return as that key so in the end we just need to
xor back our cipher using key to get the plain text
'''
# xor to bytes and return
def xor_me(bt1, bt2):
    result = b''
    
    for i in range(len(bt1)):
        result += bytes([bt1[i] ^ bt2[i]])
    
    return result

#  remove empty bytes
def normalize(bt):
    count = len(bt)
    while bt[count - 1] == 0:
        count -= 1
    return bt[0: count]

def crack_by_edit(cipher: bytes, edit_oracle):
    # get edit cipher of same size
    edit_cipher = edit_oracle(cipher, 0, bytes([0]) * len(cipher))
    # xor it back
    plain = xor_me(cipher, edit_cipher)
    # remove some last bytes
    return normalize(plain)


def test_3():
    o = oracle_3()
    for t in test_inputs:
        t = t.encode('utf-8')
        c = o.encrypt(t)
        
        p = crack_by_edit(c,o.edit)
        print(p == t)
        
    pass

if __name__ == '__main__':
    test_3()

    

True
True


(*25 Points*) Challenge 4: Breaking SHA-1 keyed mac
-------------------------------------

### Description

We have discussed about hash mac in class.  
Here is an example of why we should follow such a standard, instead of using something that *looks* secure.

`sha1` is an implementation of `sha1` hash in python, and using it, you could end up with the following scheme to generate MAC.

```
SHA1(key || original-message)
```

As you may notice in the code (or [the description of the algorithm](https://en.wikipedia.org/wiki/SHA-1)),  
SHA1 pads the input to make its length be a multiple of block size, and it computes the new hash incrementally. i.e.,
SHA1(msg1 || msg2) can be computed without msg1, if the length of msg1 is multiple of the block size and you know SHA1(msg1).




In [111]:
# https://en.wikipedia.org/wiki/SHA-1
def sha1(msg: bytes, 
    h0 = 0x67452301,
    h1 = 0xEFCDAB89,
    h2 = 0x98BADCFE,
    h3 = 0x10325476,
    h4 = 0xC3D2E1F0,
    padder = None
):
    if(padder == None):
        ml = len(msg)
        len_pad = 56 - (ml % 64)
        if len_pad <= 0: len_pad += 64

        pad = bytes([0x80] + [0] * (len_pad-1))
        padded = msg + pad + (ml*8).to_bytes(8,'big')
    else:
        padded = padder(msg)


    def left_rotate(n, a):
        return ((n << a) | (n >> (32 - a))) & 0xffffffff

    for cnk in chunks(padded,64):
        w = list(map(lambda x: int.from_bytes(x,'big'),chunks(cnk,4)))
        w = w * 5
        for i in range(16,80):
            w[i] = left_rotate((w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]),1)

        a, b, c, d, e = h0, h1, h2, h3, h4
        for i in range(80):
            if 0 <= i <= 19:
                f = (b & c) | (~b & d)
                #f = d ^ (b & ( c ^ d))
                k = 0x5A827999
            elif 20 <= i <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= i <= 59:
                f = (b & c) | (b & d) | (c & d)
                k = 0x8F1BBCDC
            elif 60 <= i <= 79:
                f = b ^ c ^ d
                k = 0xCA62C1D6
            
            tmp = (left_rotate(a,5) + f + e + k + w[i]) & 0xFFFFFFFF
            e = d
            d = c
            c = left_rotate (b,30)
            b = a
            a = tmp
        h0 = (h0 + a) & 0xFFFFFFFF
        h1 = (h1 + b) & 0xFFFFFFFF
        h2 = (h2 + c) & 0xFFFFFFFF
        h3 = (h3 + d) & 0xFFFFFFFF
        h4 = (h4 + e) & 0xFFFFFFFF
    
    hh = [h0,h1,h2,h3,h4]
    hh = list(map(lambda x: x.to_bytes(4,'big'),hh))
    res = bytes([])
    for h in hh:
        res += h
    #print(format(int.from_bytes(res,'big'),'x'))

    return res

msg = b'username=hyungon;user=yes0000000'

def get_oracle_5():
    key = token_bytes(24)
    def sign(msg: bytes):
        return sha1(key + msg)
        
    def verify(msg: bytes, mac: bytes):
        ref = sign(msg)
        return ref == mac
    
    return sign, verify


if __name__ == '__main__':
    s,v = get_oracle_5()
    mac = s(msg)
    print(v(msg,mac))




True


### Task

Here your task is to append a message of your choice to a given mac and original data without the key, and get it passed the `verify` function.  
Implement `forge_sha1` doing this.  

In [148]:
'''
In this task we will use lenth extension attack described from the lecture notes
The idea we need to make new message in the following format -> {K}{filling_bytes}{MSG}
For the key we don't need to know it, since the propery of hash function let us just to pass
current MAC, so it will continue working
'''
def extract_states(mac):
    hh = [b'', b'', b'', b'', b'']
    for i in range(len(mac)): # split bytes
        hh[int(i / 4)] += bytes([mac[i]])
    for i in range(len(hh)): # to int
        hh[i] = int.from_bytes(hh[i], "big")
    return hh

# our hashing algorithm supports custom padding function
custom_len = "I will be global"
def custom_padder(msg): #from sha1 implementation
    ml = len(msg)
    if (custom_len != 0): ml = custom_len
    len_pad = 56 - (ml % 64)
    if len_pad <= 0: len_pad += 64
    pad = bytes([0x80] + [0] * (len_pad-1))
    padded = msg + pad + (ml*8).to_bytes(8,'big')
    return padded

def forge_sha1(original_data: bytes, mac: bytes, new_data_to_append: bytes, key_size):
    global custom_len
    custom_len = 0
    # hh states from mac, they continue working
    hh_states = extract_states(mac)
    # pad initial padd of size msg and key
    # we will use this part to hash new message so our added new data will be hashed indepently
    # and key part will be hashed as it was given key
    filling_padded = padder(bytes([0])*(key_size) + original_data)
    # use custom len to overwrite ml
    custom_len = len(filling_padded) + len(new_data_to_append)
    forged_message = filling_padded[key_size:] + new_data_to_append
    forged_mac = sha1(new_data_to_append, 
                      hh_states[0], 
                      hh_states[1], 
                      hh_states[2], 
                      hh_states[3], 
                      hh_states[4], 
                      custom_padder)
    return forged_message, forged_mac

def test_4():
    orig_msg = b'username=hyungon;user=yes'
    msg_to_append = b';admin=true;run_sudo_script=\"echo "sorry" && rm -rf\"/* "'
    s,v = get_oracle_5()
    orig_mac = s(orig_msg)
    passed_key_size = 0
    for i in [16,24,32,48]:
        new_msg, new_mac = forge_sha1(orig_msg, orig_mac, msg_to_append,i)
        if v(new_msg, new_mac):
            passed_key_size = i
            break
    
    if(passed_key_size == 0):
        print('FAILED')
    else:
        print(passed_key_size)
        print('PASSED')

    pass

if __name__ == '__main__':
    test_4()
    pass


24
PASSED


.(*25 Points*) Challenge 5: Breaking Sha-1 HMAC
----------------------------------------------

### Description

HMAC is considered secure in that it is free from the attacks like we've done in Challenge 5.  
However, some small implementation details could make it insecure.  
Please check the `get_oracle_5` that implements such an (artificially) insecure HMAC.



In [11]:
def get_oracle_5():
    block_size = 16
    insecure_delay = 0.05
    key = int.from_bytes(token_bytes(16),'big')

    
    opad = key ^ int.from_bytes(bytes([0x5c]) * block_size, 'big')
    ipad = key ^ int.from_bytes(bytes([0x36]) * block_size, 'big')

    key = key.to_bytes(16,'big')
    opad = opad.to_bytes(16,'big')
    ipad = ipad.to_bytes(16,'big')

    def sign(msg: bytes):
        return sha1(opad + sha1(ipad + msg))

    def insecure_eq(a: bytes, b: bytes):
        if len(a) != len(b): return False
        for i in range(len(a)):
            if a[i] != b[i]: return False
            sleep(insecure_delay)
        return True
    def verify(msg: bytes, mac:bytes):
        ref = sign(msg)
        if insecure_eq(ref,mac): return True
        else: return False
    
    return verify, sign






### Task

Assume an attacker can repeatedly call `verify`. `sign` is just for debugging.
Leveraging the existence of `insecure_eq`, please write a function that obtains a valid mac for
and given message by calling only the `verify`.

#### Strategy

- Measure the execution time of `verify` function calls.



In [12]:
def forge_hmac_timing(msg: bytes, verify_oracle, len_mac = 20):

    return b'0'

def test_5():
    msg = b'username=hyungon;admin=yes'
    v, s = get_oracle_5()
    learned = forge_hmac_timing(msg, v)
    print(v(msg,bytes(learned)))
    print(bytes(learned).hex())
    pass

if __name__ == '__main__':
    test_5()
    pass



False
30


(*20 Points*) Challenge 6: Implement RSA
-------------------------------------


### Description

Unlike eariler challenges, we'll be first implementing our own oracle for the next challenge.
`oracle_7` is a work-in-progress, which is supposed to give you and access to `encrypt` and `decrypt` functions.

### Task

Please complete the oracle by implementing `encrypt`, `decrypt`, and `__generate_arguments` functions.  
Note that the oracle takes `e`, the exponent as an input for the next challenge.
Assume that the plaintext is shorter than the key size.

### Notes
l, which is typically referred to as lambda, should be lcm(p-1,q-1) in our case. Please refer to "Key generation" part of https://en.wikipedia.org/wiki/RSA_(cryptosystem).



In [13]:



# insecure "textbook" rsa
class oracle_7():
    
    def __init__(self, e: int):
        self.e = e
        self.__generate_arguments()

    def __get_primes(self,key_length):
        rg = Random.new().read
        rg(int.from_bytes(token_bytes(1),'big'))
        keypair = RSA.generate(key_length, rg)
        return keypair.p, keypair.q
    def egcd(self, a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = self.egcd(b % a, a)
            return (g, x - (b // a) * y, y)

    def __egcd(self,a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = self.egcd(b % a, a)
            return (g, x - (b // a) * y, y)

    def __modinv(self,a, m):
        g, x, y = self.egcd(a, m)
        if g != 1:
            raise Exception('modular inverse does not exist')
        else:
            return x % m

    def __generate_arguments(self):
        
        self.p = 0
        self.q = 0
        self.n = 0
        self.l = 0
        self.d = 0
        self.pubkey = 0,0
        self.privkey = 0,0
        pass

    def get_pubkey(self):
        return self.pubkey

    def encrypt(self,msg: bytes):
        return b'0'

    def decrypt(self,msg: bytes):
        return b'0'

def test_7():
    o = oracle_7(65537)
    msg = b'''
    Computer security, cybersecurity[1] or information technology security (IT security) is the protection of computer systems and networks from the theft of or damage to their hardware, software, or electronic data, as well as from the disruption or misdirection of the services they provide.
    '''
    c = o.encrypt(msg)
    p = o.decrypt(c)
    print(p == msg)

if __name__ == '__main__':
    test_7()



False


(*30 Points) Challenge 7. Break RSA with e=3
-------------------------------------------

### Description

Like many other cryptographic schemes, RSA is secure only if implemented and used under the standard.  
A common misimplementation of RSA is to choose the exponent `e` to be `3`, to make the encryption faster. It could be helpful if the computation power of the encryper side is expected to be weak.

### Task

Please fill out `crack_rsa` that takes three ciphertexts, each of which is generated from one plaintest, and the corresponding oracles (public keys).
For the strategy, please refer to [https://cryptopals.com/sets/5/challenges/40].


In [14]:

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def crack_rsa(ciphers, ns):
    m = 0
    return m
    

    pass
def test():
    o = [oracle_7(3),oracle_7(3),oracle_7(3)]
    msg = b'I hate RSA' * 10
    ciphers = list(map(lambda x: int.from_bytes(x.encrypt(msg),'big'), o ))
    ns = list(map(lambda x: x.n, o ))
    print(ciphers)
    msg = b'I hate RSA' * 10
    p = crack_rsa(ciphers,ns)
    print(p)
if __name__ == '__main__':
    test()

[b'0', b'0', b'0']
False


# This is the end of lab 1.